In [1]:
import tensorflow as tf

2023-07-30 01:20:23.582975: I tensorflow/core/util/util.cc:169] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.


In [2]:
cancer_gen_expr_model = tf.keras.models.load_model("Models//cancer_gen_expr_model")
cancer_gen_mut_model = tf.keras.models.load_model("Models//cancer_gen_mut_model")
cancer_dna_methy_model = tf.keras.models.load_model("Models//cancer_dna_methy_model")
cancer_gen_cnv_model = tf.keras.models.load_model("Models//cancer_gen_cnv_model")

2023-07-30 01:20:25.360252: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  SSE4.1 SSE4.2 AVX AVX2 AVX512F AVX512_VNNI FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


2023-07-30 01:20:25.858317: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1532] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 78910 MB memory:  -> device: 0, name: NVIDIA A100 80GB PCIe, pci bus id: 0000:31:00.0, compute capability: 8.0


In [3]:
cancer_gen_expr_model.trainable = False
cancer_gen_mut_model.trainable = False
cancer_dna_methy_model.trainable = False
cancer_gen_cnv_model.trainable = False

In [4]:
import pickle

In [5]:
with open("csa_data//drug_features.pickle", "rb") as f:
    dict_features = pickle.load(f)

In [6]:
with open("csa_data//norm_adj_mat.pickle", "rb") as f:
    dict_adj_mat = pickle.load(f)

In [7]:
with open("csa_data//combo_gen_expr_cnv_norm.pickle", "rb") as f:
    combo_gen_expr_cnv_norm = pickle.load( f)

In [8]:
common_achs = list(combo_gen_expr_cnv_norm.keys())

In [9]:
# combo_gen_expr_cnv_norm

In [10]:
import pandas as pd

In [11]:
pubchem_to_drugs_df = pd.read_csv('csa_data/1.Drug_listMon Jun 24 09_00_55 2019.csv')

In [12]:
# pubchem_to_drugs_df

In [13]:
# pubchem_to_drugs_df

In [14]:
pubchem_to_drugs_df = pubchem_to_drugs_df[["drug_id", "PubCHEM"]]

In [15]:
import numpy as np

In [16]:
pubchem_to_drugs_df["PubCHEM"] = [int(val) if str(val).isdigit() else np.nan for val in pubchem_to_drugs_df["PubCHEM"] ]

In [17]:
pubchem_to_drugs_df = pubchem_to_drugs_df.dropna()

In [18]:
pubchem_to_drugs_df["Drug_ID"] = pubchem_to_drugs_df["drug_id"].astype(str)

In [19]:
pubchem_to_drugs_df["Drug_ID"] = ["Drug_" + str(i) for i in pubchem_to_drugs_df["drug_id"].values]

In [20]:
pubchem_to_drugs_df.head()

,drug_id,PubCHEM,Drug_ID
0,1242,9863776.0,Drug_1242
1,179,3385.0,Drug_179
2,86,10172943.0,Drug_86
3,55,9549184.0,Drug_55
4,1001,65110.0,Drug_1001


In [21]:
dualgcn_train = pd.read_csv("csa_data//DualGCN_Embedding_train.csv")

In [22]:
dualgcn_train.shape

(69214, 388)

In [23]:
dualgcn_test = pd.read_csv("csa_data//DualGCN_Embedding_test.csv")

In [24]:
dualgcn_test.shape

(17316, 388)

In [25]:
# pubchem_to_drugs_df[['Drug_ID', 'PubCHEM']].dtypes

In [26]:
train_keep = dualgcn_train[['Cell_Line', 'Drug_ID']].merge(pubchem_to_drugs_df[['Drug_ID', 'PubCHEM']], left_on = 'Drug_ID', right_on = 'PubCHEM')

In [27]:
test_keep = dualgcn_test[['Cell_Line', 'Drug_ID']].merge(pubchem_to_drugs_df[['Drug_ID', 'PubCHEM']], left_on = 'Drug_ID', right_on = 'PubCHEM')

In [28]:
train_keep = train_keep.iloc[:, [0,2]]

In [29]:
train_keep.columns = ['Cell_Line', 'Drug_ID']

In [30]:
train_keep.head()

,Cell_Line,Drug_ID
0,ACH-000070,Drug_1
1,ACH-000105,Drug_1
2,ACH-000981,Drug_1
3,ACH-000061,Drug_1
4,ACH-001106,Drug_1


In [31]:
train_keep['Drug_ID'].nunique()

208

In [32]:
train_keep['Cell_Line'].nunique()

525

In [33]:
train_keep = train_keep[train_keep['Cell_Line'].isin(common_achs)]

In [34]:
train_keep.shape

(68518, 2)

In [35]:
test_keep = test_keep.iloc[:, [0,2]]

In [36]:
# test_keep['Drug_ID'].nunique()

In [37]:
test_keep.columns = ['Cell_Line', 'Drug_ID']

In [38]:
test_keep.shape

(17316, 2)

In [39]:
test_keep['Cell_Line'].nunique()

524

In [40]:
test_keep = test_keep[test_keep['Cell_Line'].isin(common_achs)]

In [41]:
test_keep.shape

(17156, 2)

In [42]:
drug_cell_ic50 = pd.read_csv("csa_data/drugs_cell_lines_ic50.csv")

In [43]:
import improve_utils as iu

In [44]:
drug_cell_ic50["Drug_ID"] = ["Drug_" + str(id1) for id1 in drug_cell_ic50["drug_id"]]

In [45]:
drug_cell_ic50

,drug_id,Cancer_Cell_Line,IC50,Drug_ID
0,1001,ACH-002137,7.258918,Drug_1001
1,1004,ACH-002137,-3.802467,Drug_1004
2,1005,ACH-002137,4.146364,Drug_1005
3,1006,ACH-002137,3.171367,Drug_1006
4,1007,ACH-002137,-4.959442,Drug_1007
...,...,...,...,...
208729,1377,ACH-000828,-0.308269,Drug_1377
208730,1378,ACH-000828,7.123334,Drug_1378
208731,1494,ACH-000828,1.442751,Drug_1494
208732,1495,ACH-000828,6.341509,Drug_1495


In [46]:
overall_df = drug_cell_ic50[['Cancer_Cell_Line', 'Drug_ID', 'IC50']]

In [47]:
overall_df.columns = ['Cell_Line', 'Drug_ID', "IC_50"]

In [48]:
overall_df.shape

(208734, 3)

In [49]:
len(set(train_keep['Drug_ID'].values).intersection(overall_df['Drug_ID'].values))

208

In [50]:
len(set(train_keep['Cell_Line'].values).intersection(overall_df['Cell_Line'].values))

519

In [51]:
# overall_df['Drug_ID'].values

In [52]:
train_keep1 = train_keep.merge(overall_df)

In [53]:
train_keep1.shape

(68518, 3)

In [54]:
test_keep1 = test_keep.merge(overall_df)

In [55]:
test_keep1.shape

(17156, 3)

In [56]:
samp_drug = test_keep1["Drug_ID"].unique()[-1]

In [57]:
samp_ach = np.array(test_keep1["Cell_Line"].unique()[-1])

In [58]:
train_keep1.to_csv("csa_data/dualgcndeepcdr_combo_train.csv", index = False)

In [59]:
test_keep1.to_csv("csa_data/dualgcndeepcdr_combo_test.csv", index = False)

In [60]:
# train_methy = cancer_dna_methy_model.predict(train_keep1["Cell_Line"].values, batch_size = 512)

In [61]:
# train_methy.shape

In [62]:
train_gcn_feats = []
train_adj_list = []
for drug_id in train_keep1["Drug_ID"].values:
    train_gcn_feats.append(dict_features[drug_id])
    train_adj_list.append(dict_adj_mat[drug_id])

In [63]:
valid_gcn_feats = []
valid_adj_list = []
for drug_id in test_keep1["Drug_ID"].values:
    valid_gcn_feats.append(dict_features[drug_id])
    valid_adj_list.append(dict_adj_mat[drug_id])

In [64]:
train_gcn_feats = np.array(train_gcn_feats).astype("float32")
valid_gcn_feats = np.array(valid_gcn_feats).astype("float32")

In [65]:
valid_gcn_feats.shape

(17156, 223, 75)

In [ ]:
train_adj_list = np.array(train_adj_list).astype("float32")
valid_adj_list = np.array(valid_adj_list).astype("float32")

In [ ]:
valid_adj_list.shape

In [ ]:
# combo_gen_expr_cnv_norm

In [ ]:
# valid_gcn_feats_omics

In [ ]:
train_gcn_feats_omics = []
for cnc_id in train_keep1["Cell_Line"].values:
    train_gcn_feats_omics.append(combo_gen_expr_cnv_norm[cnc_id])

In [ ]:
train_gcn_feats_omics = np.array(train_gcn_feats_omics).astype("float32")

In [ ]:
train_gcn_feats_omics.shape

In [ ]:
valid_gcn_feats_omics = []
for cnc_id in test_keep1["Cell_Line"].values:
    valid_gcn_feats_omics.append(combo_gen_expr_cnv_norm[cnc_id])

In [ ]:
valid_gcn_feats_omics = np.array(valid_gcn_feats_omics).astype("float32")

In [ ]:
valid_gcn_feats_omics.shape

In [ ]:
# train_methy = cancer_dna_methy_model(train_keep1["Cell_Line"].values)

In [ ]:
# cancer_gen_expr_model(samp_ach).numpy().shape[0]

In [ ]:
from tensorflow.keras import backend as K

In [ ]:
# cancer_gen_expr_model(train_keep1["Cell_Line"].values[:5])

In [ ]:
training = False
dropout1 = 0.20
dropout2 = 0.30

In [ ]:
## get the model architecture
def deepcdrgcn(training = training, dropout1 = dropout1, dropout2 = dropout2):
    
    input_gcn_features = tf.keras.layers.Input(shape = (dict_features[samp_drug].shape[0], 75))
    input_norm_adj_mat = tf.keras.layers.Input(shape = (dict_adj_mat[samp_drug].shape[0], dict_adj_mat[samp_drug].shape[0]))
    mult_1 = tf.keras.layers.Dot(1)([input_norm_adj_mat, input_gcn_features])
    dense_layer_gcn = tf.keras.layers.Dense(256, activation = "relu")
    dense_out = dense_layer_gcn(mult_1)
    dense_out = tf.keras.layers.BatchNormalization()(dense_out)
    dense_out = tf.keras.layers.Dropout(dropout1)(dense_out, training = training)
    mult_2 = tf.keras.layers.Dot(1)([input_norm_adj_mat, dense_out])
    dense_layer_gcn = tf.keras.layers.Dense(256, activation = "relu")
    dense_out = dense_layer_gcn(mult_2)
    dense_out = tf.keras.layers.BatchNormalization()(dense_out)
    dense_out = tf.keras.layers.Dropout(dropout1)(dense_out, training = training)

    dense_layer_gcn = tf.keras.layers.Dense(100, activation = "relu")
    mult_3 = tf.keras.layers.Dot(1)([input_norm_adj_mat, dense_out])
    dense_out = dense_layer_gcn(mult_3)
    dense_out = tf.keras.layers.BatchNormalization()(dense_out)
    dense_out = tf.keras.layers.Dropout(dropout1)(dense_out, training = training)

    dense_out = tf.keras.layers.GlobalAvgPool1D()(dense_out)
    
    input_gen_expr = tf.keras.layers.Input(shape = (cancer_gen_expr_model(samp_ach).numpy().shape[0],2))
    
    l1 = tf.keras.layers.Dense(32)(input_gen_expr)
    l1 = tf.keras.layers.Dropout(dropout1)(l1, training = training)
    l2 = tf.keras.layers.Dense(128)(l1)
    l2 = tf.keras.layers.Dropout(dropout1)(l2, training = training)
    
    dense_layer_gcn1 = tf.keras.layers.Dense(256, activation = "relu")
    dense_out1 = dense_layer_gcn1(l2)
    dense_out1 = tf.keras.layers.BatchNormalization()(dense_out1)
    dense_out1 = tf.keras.layers.Dropout(dropout1)(dense_out1, training = training)
# mult_21 = tf.keras.layers.Dot(1)([const_input, dense_out1])
    dense_layer_gcn1 = tf.keras.layers.Dense(256, activation = "relu")
    dense_out1 = dense_layer_gcn1(dense_out1)
    dense_out1 = tf.keras.layers.BatchNormalization()(dense_out1)
    dense_out1 = tf.keras.layers.Dropout(dropout1)(dense_out1, training = training)
    dense_layer_gcn1 = tf.keras.layers.Dense(256, activation = "relu")
    dense_out1 = dense_layer_gcn1(dense_out1)
    dense_out1 = tf.keras.layers.BatchNormalization()(dense_out1)
    dense_out1 = tf.keras.layers.Dropout(dropout1)(dense_out1, training = training)
    dense_layer_gcn1 = tf.keras.layers.Dense(256, activation = "relu")
    dense_out1 = dense_layer_gcn1(dense_out1)
    dense_out1 = tf.keras.layers.BatchNormalization()(dense_out1)
    dense_out1 = tf.keras.layers.Dropout(dropout1)(dense_out1, training = training)
    dense_out1 = tf.keras.layers.GlobalAvgPool1D()(dense_out1)
    
    
    input_gen_methy1 = tf.keras.layers.Input(shape = (1,), dtype = tf.string)
    # cancer_dna_methy_model.trainable
    input_gen_methy = cancer_dna_methy_model(input_gen_methy1)
    input_gen_methy.trainable = False
    gen_methy_layer = tf.keras.layers.Dense(64, activation = "tanh")
    
    gen_methy_emb = gen_methy_layer(input_gen_methy)
    gen_methy_emb = tf.keras.layers.BatchNormalization()(gen_methy_emb)
    gen_methy_emb = tf.keras.layers.Dropout(dropout1)(gen_methy_emb, training = training)
    gen_methy_layer = tf.keras.layers.Dense(32, activation = "relu")
    gen_methy_emb = gen_methy_layer(gen_methy_emb)
    
    
    input_gen_mut1 = tf.keras.layers.Input(shape = (1,), dtype = tf.string)
    input_gen_mut = cancer_gen_mut_model(input_gen_mut1)
    input_gen_mut.trainable = False
    
    reshape_gen_mut = tf.keras.layers.Reshape((1, cancer_gen_mut_model(samp_ach).numpy().shape[0], 1))
    reshape_gen_mut = reshape_gen_mut(input_gen_mut)
    gen_mut_layer = tf.keras.layers.Conv2D(50, (1, 700), strides=5, activation = "tanh")
    gen_mut_emb = gen_mut_layer(reshape_gen_mut)
    pool_layer = tf.keras.layers.MaxPooling2D((1,5))
    pool_out = pool_layer(gen_mut_emb)
    gen_mut_layer = tf.keras.layers.Conv2D(30, (1, 5), strides=2, activation = "relu")
    gen_mut_emb = gen_mut_layer(pool_out)
    pool_layer = tf.keras.layers.MaxPooling2D((1,10))
    pool_out = pool_layer(gen_mut_emb)
    flatten_layer = tf.keras.layers.Flatten()
    flatten_out = flatten_layer(pool_out)
    all_omics = tf.keras.layers.Concatenate()([gen_methy_emb, flatten_out, dense_out1, dense_out])
    x = tf.keras.layers.Dense(300,activation = 'tanh')(all_omics)
    x = tf.keras.layers.Dropout(dropout1)(x, training = training)
    x = tf.keras.layers.Lambda(lambda x: K.expand_dims(x,axis=-1))(x)
    x = tf.keras.layers.Lambda(lambda x: K.expand_dims(x,axis=1))(x)
    x = tf.keras.layers.Conv2D(filters=30, kernel_size=(1,150),strides=(1, 1), activation = 'relu',padding='valid')(x)
    x = tf.keras.layers.MaxPooling2D(pool_size=(1,2))(x)
    x = tf.keras.layers.Conv2D(filters=10, kernel_size=(1,5),strides=(1, 1), activation = 'relu',padding='valid')(x)
    x = tf.keras.layers.MaxPooling2D(pool_size=(1,3))(x)
    x = tf.keras.layers.Conv2D(filters=5, kernel_size=(1,5),strides=(1, 1), activation = 'relu',padding='valid')(x)
    x = tf.keras.layers.MaxPooling2D(pool_size=(1,3))(x)
    x = tf.keras.layers.Dropout(dropout1)(x, training = training)
    x = tf.keras.layers.Flatten()(x)
    x = tf.keras.layers.Dropout(dropout2)(x, training = training)
    final_out_layer = tf.keras.layers.Dense(1)
    final_out = final_out_layer(x)
    simplecdr = tf.keras.models.Model([input_gcn_features, input_norm_adj_mat, input_gen_expr,
                                   input_gen_methy1, input_gen_mut1], final_out)
    simplecdr.compile(loss = tf.keras.losses.MeanSquaredError(), 
                      # optimizer = tf.keras.optimizers.Adam(lr=1e-3),
                    optimizer = tf.keras.optimizers.Adam(lr=0.001, beta_1=0.9, beta_2=0.999, epsilon=None, decay=0.0, amsgrad=False), 
                    metrics = [tf.keras.metrics.RootMeanSquaredError()])
    
    return simplecdr

In [ ]:
check = deepcdrgcn(training = training, dropout1 = dropout1, dropout2 = dropout2)

In [ ]:
check.summary()

In [ ]:
# [layer.name for layer in check.layers]

In [ ]:
check.fit([train_gcn_feats, train_adj_list, train_gcn_feats_omics, train_keep1["Cell_Line"].values.reshape(-1,1), train_keep1["Cell_Line"].values.reshape(-1,1)],
          train_keep1["IC_50"].values.reshape(-1,1),
         validation_data = ([valid_gcn_feats, valid_adj_list, valid_gcn_feats_omics, test_keep1["Cell_Line"].values.reshape(-1,1), test_keep1["Cell_Line"].values.reshape(-1,1)],
          test_keep1["IC_50"].values.reshape(-1,1)), 
         batch_size = 64, epochs = 10000, 
         callbacks = tf.keras.callbacks.EarlyStopping(monitor = "val_loss", patience = 10, restore_best_weights=True, 
                                                     mode = "min"), shuffle = True, verbose = 1, 
         validation_batch_size = 64)

In [ ]:
check.save("Models/combo_cdr_gcn_on_ftp_data_drop_on_train")